In [ ]:
!pip install pyspark

In [ ]:
# Solo se necesitan si se corre con spark-submit (o si se corre en Google Colab)
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ejemplo4').getOrCreate()

In [ ]:
df = spark.read.csv('/content/datos_null.csv', header=True, inferSchema=True)
df.show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|  null|
|emp2|  null|  null|
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Eliminar todas las filas que tienen datos faltantes
df.na.drop().show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp3| Mario| 560.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Eliminar las filas que tienen menos de 3 valores no nulo
df.na.drop(thresh=3).show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp3| Mario| 560.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Eliminar las filas que tienen menos de 2 valores no nulos
df.na.drop(thresh=2).show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|  null|
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Eliminar las filas que tienen menos de 1 valor no nulo
df.na.drop(thresh=1).show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|  null|
|emp2|  null|  null|
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
df.na.fill("NULO").show()
df.na.fill(0).show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|  null|
|emp2|  NULO|  null|
|emp3| Mario| 560.0|
|emp4|  NULO| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|   0.0|
|emp2|  null|   0.0|
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Eliminar valores nulos en una columna determinada
df.na.drop(subset=["Ventas"]).show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Elimina si hay algún nulo
df.na.drop(how='any').show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp3| Mario| 560.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Elimina solo si todos los valores son nulos
df.na.drop(how='all').show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|  null|
|emp2|  null|  null|
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



## Imputación de valores

In [ ]:
# Usando el mismo valor (depende del tipo de valor)
df.na.fill("Nuevo Valor").show()

+----+-----------+------+
|  ID|     Nombre|Ventas|
+----+-----------+------+
|emp1|       Juan|  null|
|emp2|Nuevo Valor|  null|
|emp3|      Mario| 560.0|
|emp4|Nuevo Valor| 345.0|
|emp5|      Carla| 456.0|
|emp6|      Diana| 320.5|
+----+-----------+------+



In [ ]:
df.na.fill(10).show()

+----+------+------+
|  ID|Nombre|Ventas|
+----+------+------+
|emp1|  Juan|  10.0|
|emp2|  null|  10.0|
|emp3| Mario| 560.0|
|emp4|  null| 345.0|
|emp5| Carla| 456.0|
|emp6| Diana| 320.5|
+----+------+------+



In [ ]:
# Se puede especificar en qué columna se desea imputar el valor
df.na.fill("Sin nombre", subset=['Nombre']).show()

+----+----------+------+
|  ID|    Nombre|Ventas|
+----+----------+------+
|emp1|      Juan|  null|
|emp2|Sin nombre|  null|
|emp3|     Mario| 560.0|
|emp4|Sin nombre| 345.0|
|emp5|     Carla| 456.0|
|emp6|     Diana| 320.5|
+----+----------+------+



In [ ]:
# Imputación usando la media
from pyspark.sql.functions import mean

# Media de la columna "ventas"
media_val = df.select(mean(df['Ventas'])).collect()

print(media_val)
print(media_val[0])
print(media_val[0][0])
# Valor de la media
media_ventas = media_val[0][0]

[Row(avg(Ventas)=420.375)]
Row(avg(Ventas)=420.375)
420.375


In [ ]:
# Imputación usando la media
df.na.fill(media_ventas, ['Ventas']).show()

+----+------+-------+
|  ID|Nombre| Ventas|
+----+------+-------+
|emp1|  Juan|420.375|
|emp2|  null|420.375|
|emp3| Mario|  560.0|
|emp4|  null|  345.0|
|emp5| Carla|  456.0|
|emp6| Diana|  320.5|
+----+------+-------+

